# DOA with 1 degree resolution

In [1]:
from preprocessing import *
from training import create_model, evaluate_model
from sklearn.preprocessing import OneHotEncoder
import tensorflow as tf
from keras import Sequential
from keras.models import Model
from keras.layers import Input
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Dropout
from keras.layers import Conv1D
from keras.layers import MaxPooling1D

In [2]:
# Label resolution of classification
RESOLUTION = 1

# Number of samples to include while creating one ML feature
SAMPLES = 2048

# Determines the overlap of samples between consecutive features
STEP = 1024

### Create training and testing sets

In [7]:
df_train = create_dataframe('train', samples=SAMPLES, step=STEP, resolution=RESOLUTION)
print()
df_test = create_dataframe('test', samples=SAMPLES, step=STEP, resolution=RESOLUTION)
print()

# Create numpy arrays with observations and one-hot labels
encoder = OneHotEncoder(handle_unknown='ignore', sparse=False)
X_train, y_train, X_test, y_test = create_whole_dataset(df_train, df_test, encoder)

np.shape(X_train), np.shape(X_test), np.shape(y_train), np.shape(y_test)

train file 3/3
test file 3/3


((582, 15, 13), (196, 15, 13), (582, 1), (196, 1))

### Fit and evaluate model

In [8]:
# Transpose the observations because Conv1D requires timesteps as the 1st dim
if X_train.shape[1] == MIC_COMBS:
    X_train, X_test = np.transpose(X_train, axes=[0, 2, 1]), np.transpose(X_test, axes=[0, 2, 1])
X_train.shape, X_test.shape

((582, 13, 15), (196, 13, 15))

In [5]:
model = create_model(X_train, y_train, X_test, y_test)

Epoch 1/20
19/19 [==============================] - 0s 2ms/step - loss: 0.0000e+00 - accuracy: 1.0000
Epoch 2/20
19/19 [==============================] - 0s 2ms/step - loss: 0.0000e+00 - accuracy: 1.0000
Epoch 3/20
19/19 [==============================] - 0s 2ms/step - loss: 0.0000e+00 - accuracy: 1.0000
Epoch 4/20
19/19 [==============================] - 0s 2ms/step - loss: 0.0000e+00 - accuracy: 1.0000
Epoch 5/20
19/19 [==============================] - 0s 2ms/step - loss: 0.0000e+00 - accuracy: 1.0000
Epoch 6/20
19/19 [==============================] - 0s 2ms/step - loss: 0.0000e+00 - accuracy: 1.0000
Epoch 7/20
19/19 [==============================] - 0s 2ms/step - loss: 0.0000e+00 - accuracy: 1.0000
Epoch 8/20
19/19 [==============================] - 0s 2ms/step - loss: 0.0000e+00 - accuracy: 1.0000
Epoch 9/20
19/19 [==============================] - 0s 2ms/step - loss: 0.0000e+00 - accuracy: 1.0000
Epoch 10/20
19/19 [==============================] - 0s 2ms/step - loss: 0.0000e+0

In [6]:
# Test model
accuracy = evaluate_model(model, X_test, y_test)
print(f'Accuracy: {accuracy}')

7/7 [==============================] - 0s 858us/step - loss: 0.0000e+00 - accuracy: 1.0000
Accuracy: 1.0
